# Week 2 — LoRA Fine-Tuning of SLM
Align Small Language Model embeddings with Diffusion CLIP Encoder.

In [1]:
import diffusers, huggingface_hub, torch
print("diffusers:", diffusers.__version__)
print("huggingface_hub:", huggingface_hub.__version__)
print("Torch CUDA available:", torch.cuda.is_available())


/home/cse-sdpl/anaconda3/envs/gpu_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


diffusers: 0.26.3
huggingface_hub: 0.20.3
Torch CUDA available: True


In [2]:
!pip install peft==0.10.0 transformers==4.37.2 accelerate==0.29.3 -q

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from diffusers import StableDiffusionPipeline
from peft import LoraConfig, get_peft_model, TaskType
import torch, pandas as pd
from tqdm import tqdm

/home/cse-sdpl/anaconda3/envs/gpu_env/lib/python3.10/site-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/home/cse-sdpl/anaconda3/envs/gpu_env/lib/python3.10/site-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [4]:
slm_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(slm_name)
slm = AutoModelForCausalLM.from_pretrained(slm_name, torch_dtype=torch.float16).to("cuda")
pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16).to("cuda")
clip_encoder = pipe.text_encoder
print("SLM & Diffusion Model Loaded Successfully")

Loading pipeline components...: 100%|██████████| 7/7 [00:01<00:00,  3.88it/s]


SLM & Diffusion Model Loaded Successfully


In [6]:
data = {"caption": [
    "a sunset over the ocean",
    "a cat sitting on a laptop",
    "a futuristic city skyline",
    "a person walking in rain with umbrella",
] * 25}
df = pd.DataFrame(data)
df.to_csv("../data/mini_text_image_dataset.csv", index=False)
print("Dataset saved")

Dataset saved


In [7]:
config = LoraConfig(task_type=TaskType.CAUSAL_LM, r=8, lora_alpha=16, target_modules=["q_proj", "v_proj"], lora_dropout=0.05)
slm_lora = get_peft_model(slm, config)
slm_lora.print_trainable_parameters()

trainable params: 1,126,400 || all params: 1,101,174,784 || trainable%: 0.10229075496156657


In [8]:
def dummy_train(slm, tokenizer, df):
    optimizer = torch.optim.AdamW(slm.parameters(), lr=1e-4)
    slm.train()
    for caption in tqdm(df['caption'][:50], desc="Training"):
        inputs = tokenizer(caption, return_tensors="pt", truncation=True, max_length=128).to("cuda")
        outputs = slm(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print("LoRA Fine-tuning Complete")
dummy_train(slm_lora, tokenizer, df)

Training: 100%|██████████| 50/50 [00:02<00:00, 17.43it/s]

LoRA Fine-tuning Complete


In [9]:
output_dir = "../adapters/slm_lora_adapter/"
slm_lora.save_pretrained(output_dir)
print(f"LoRA adapter saved to {output_dir}")

LoRA adapter saved to ../adapters/slm_lora_adapter/
